In [7]:
import pandas 

dfs = pandas.read_excel('C2011-07 Patient Transport System.xlsx', sheet_name=None,
    skiprows=1)
sheetnames = list(dfs.keys())

In [35]:
def parse_date(date):
    elms = date.split()
    total_hour = 0
    for elm in elms:
        if elm.endswith("d"):
            total_hour += int(elm[:-1])*24
        elif elm.endswith("h"):
            total_hour += int(elm[:-1])
    return total_hour

def find_column_indices(header, col_names):
    indices = []
    for name in col_names:
        for i, h in enumerate(header):
            if h == name:
                indices.append(i)
    return indices

In [67]:
import os 
if not os.path.isdir("data"):
    os.makedirs("data")
baselines = dfs['Baseline Schedule'][['ID', 'Duration', 'Total Cost']].values
baselines[:,1] = [parse_date(x) for x in baselines[:,1]]

In [43]:
# planned duration
PD = baselines[0,1]
# tracking periods
tracking_periods = [x for x in sheetnames if "TP" in x]
n_tracking_periods = len(tracking_periods)
print("PD:", PD)
print("Number of tracking periods:", n_tracking_periods)
# init trend
Ts_AT = [PD/n_tracking_periods]
Ts_ES = [PD/n_tracking_periods]
print("T0_AT = T0_ES: ", Ts_AT[0])

PD: 9336
Number of tracking periods: 23
T0_AT = T0_ES:  405.9130434782609


In [75]:
# Col 0 = ID, col 12 = Duration
beta = 0.15
ATs = [0] # init AT0 = 0
t = 1
EVs = [0]
PVs = [0]
ESs = [0]
EACs = [] # predict project duration
for period in tracking_periods:
    print("Tracking periods:", period)
    cols = find_column_indices(dfs[period].values[1], ["ID", "Actual Duration", "Earned Value (EV)", "Planned Value (PV)"])
    data_period = dfs[period].values[2:, cols] 
    data_period[:,1] = [parse_date(x) for x in data_period[:,1]]
    assert (baselines[:,0] == data_period[:,0]).sum() == len(baselines), "Wrong permutation!"

    # current trend
    cur_AT = data_period[0,1]
    ATs.append(cur_AT)
    T_AT = beta*(ATs[t] - ATs[t-1]) + (1-beta)*Ts_AT[t-1]
    Ts_AT.append(T_AT)

    EV = data_period[0,2]
    PV = data_period[0,3]
    PVs.append(PV)
    EVs.append(EV)
    if PVs[t] == PVs[t-1]:
        ES = t-1
    else:
        ES = t-1 + (EV-PVs[t-1]) / (PVs[t]-PVs[t-1])
    ESs.append(ES)
    T_ES = beta*(ESs[t] - ESs[t-1]) + (1-beta)*Ts_ES[t-1]
    Ts_ES.append(T_ES)

    k = (PD-ESs[t]) / T_ES
    EAC = ATs[t] + k * T_AT
    EACs.append(EAC)
    print("EAC:", EAC, k)
    # end calculate
    t += 1
print("Project actual duration:", data_period[0,1])

Tracking periods: Project Control - TP1
EAC: 11758.123453057986 27.044167753068695
Tracking periods: TP2
EAC: 50454.39598825908 31.8166679447867
Tracking periods: TP3
EAC: 50873.246641700316 31.77735908131532
Tracking periods: TP4
EAC: 51373.20706183718 31.790190312949782
Tracking periods: TP5
EAC: 51847.907746892706 31.78678373447647
Tracking periods: TP6
EAC: 52352.11858671925 31.802347279319168
Tracking periods: TP7
EAC: 52904.19735922575 31.848681946606575
Tracking periods: TP8
EAC: 55796.28259590199 33.39925173329627
Tracking periods: TP9
EAC: 51767.90266671162 30.501112978152804
Tracking periods: TP10
EAC: 54706.18446915143 32.08137947623256
Tracking periods: TP11
EAC: 54311.39006551621 31.51903211337108
Tracking periods: TP12
EAC: 55451.99471853625 31.943691076638466
Tracking periods: TP13
EAC: 55434.71897448063 31.624025447637408
Tracking periods: TP14
EAC: 56230.708977221 31.738277705354076
Tracking periods: TP15
EAC: 56528.33373550779 31.783470880346055
Tracking periods: TP16

# Cost forecasting

In [68]:
# planned duration
BAC = baselines[0,2]
# tracking periods
tracking_periods = [x for x in sheetnames if "TP" in x]
n_tracking_periods = len(tracking_periods)
print("BAC:", BAC)
print("Number of tracking periods:", n_tracking_periods)
# init trend
Ts_AC = [BAC/n_tracking_periods]
Ts_EV = [BAC/n_tracking_periods]
print("T0_AC = T0_EV: ", Ts_AC[0])

BAC: 180759.44
Number of tracking periods: 23
T0_AC = T0_EV:  7859.106086956522


In [73]:
# Col 0 = ID, col 12 = Duration
beta = 0.15
ACs = [0] # init AT0 = 0
t = 1
EVs = [0]
EAC_costs = [] # predict project duration
for period in tracking_periods:
    print("Tracking periods:", period)
    cols = find_column_indices(dfs[period].values[1], ["ID", "Actual Cost", "Earned Value (EV)", "Planned Value (PV)"])
    data_period = dfs[period].values[2:, cols] 
    assert (baselines[:,0] == data_period[:,0]).sum() == len(baselines), "Wrong permutation!"

    # current trend
    cur_AC = data_period[0,1]
    ACs.append(cur_AC)
    T_AC = beta*(ACs[t] - ACs[t-1]) + (1-beta)*Ts_AC[t-1]
    Ts_AC.append(T_AC)

    EV = data_period[0,2]
    EVs.append(EV)
    T_EV = beta*(EVs[t] - EVs[t-1]) + (1-beta)*Ts_EV[t-1]
    Ts_EV.append(T_EV)

    k = (BAC-EVs[t]) / T_EV
    EAC = ACs[t] + k * T_AC
    EACs.append(EAC)
    print("EAC:", EAC, k)
    # end calculate
    t += 1
print("Project actual costs: ", data_period[0,1])

Tracking periods: Project Control - TP1
EAC: 180759.44 23.494837195290426
Tracking periods: TP2
EAC: 180759.44 27.6409849356358
Tracking periods: TP3
EAC: 184559.31011985693 28.447088414438838
Tracking periods: TP4
EAC: 184403.74216758274 31.77590535250537
Tracking periods: TP5
EAC: 183839.33854739068 30.164693647316167
Tracking periods: TP6
EAC: 185317.30132215022 34.48445213183372
Tracking periods: TP7
EAC: 184679.54944743039 33.35515019867455
Tracking periods: TP8
EAC: 186355.06807933128 38.81813448278941
Tracking periods: TP9
EAC: 186355.06807933128 45.66839350916401
Tracking periods: TP10
EAC: 186060.22067067312 50.105556092869655
Tracking periods: TP11
EAC: 185935.60455068896 57.146760315004805
Tracking periods: TP12
EAC: 185139.88871996186 53.702437051069225
Tracking periods: TP13
EAC: 185034.7834053899 61.07693980200441
Tracking periods: TP14
EAC: 189392.01908728524 3.9815470441422636
Tracking periods: TP15
EAC: 189392.0190872852 4.684172993108545
Tracking periods: TP16
EAC: 19